In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("train.csv")
print(df.shape)
df.head(2)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df.drop(columns=['PassengerId','Ticket', 'Name','Cabin'],inplace=True)

In [6]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                df['Survived'],
                                                test_size=0.2,
                                                random_state=42)
print(x_train.shape,x_test.shape)

(712, 7) (179, 7)


In [11]:
x_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
660,1,male,50.0,2,0,133.65,S
763,1,female,36.0,1,2,120.00,S


In [33]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_Age',SimpleImputer(),[2]),
    ('impute_Embarked',SimpleImputer(strategy='most_frequent'),[6]),
],remainder='passthrough')

# One Hot Encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

# train the model
trf5 = DecisionTreeClassifier()

In [34]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

### Pipeline Vs make_pipeline

Pipeline requires naming of steps, make_pipeline does not.

(same applies to ColumnTransformer vs make_column_transformer)

In [45]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [46]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_Age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000020A7B15A2A0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [47]:
y_pred = pipe.predict(x_test)

In [48]:
accuracy_score(y_test,y_pred)

0.6256983240223464

In [50]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

In [51]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [1]:
import pickle
import numpy as np

In [2]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [9]:
test_input2 = np.array([2,'male',31.0,1,0,10.5,'S'],dtype=object).reshape(1,7)

In [10]:
pipe.predict(test_input2)

array([0], dtype=int64)